# Articles classification model

## Libraries and parameters

In [ ]:
!pip install transformers

In [ ]:
import os
import json
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, 
    BertTokenizer, 
    BertForSequenceClassification,
    AdamW
)
from sklearn.metrics import  (
    f1_score, 
    accuracy_score, 
    multilabel_confusion_matrix, 
    confusion_matrix
)
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm.auto import tqdm

In [ ]:
VER = 'v0'
DATA_PATH = './data'
MDLS_PATH = f'/home/jovyan/__LABELING/models_{VER}'
CONFIG = {
    'data_file': 'project-6-at-2024-03-13-08-22-16c5be41.json',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'bbone': 'DeepPavlov/rubert-base-cased-sentence',
    'targets_type': 'explicit', # targets can be 'all', explicit', 'general' 
    'targets': [
        'target_0_explicit', 
        'target_3_explicit', 
        'target_3_general', 
        'target_4_explicit',
        'target_4_general',
        'target_7_explicit',
        'target_7_general', 
        'target_11_explicit', 
        'target_11_general',
        'target_12_explicit', 
        'target_12_general'
    ], 
    'targets_description': {
        'target_0': 'ЦУР отсутствуют', 
        'target_3': 'ЦУР 3 - Хорошее здоровье и благополучие',
        'target_4': 'ЦУР 4 - Качественное образование',
        'target_7': 'ЦУР 7 - Недорогостоящая и чистая энергия', 
        'target_11': 'ЦУР 11 - Устойчивые города',
        'target_12': 'ЦУР 12 - Ответственное потребление и производство', 
    },
    'folds': 5,
    'max_seq_len': 512,
    'batch_size': 32,
    'num_workers': 4,
    'acc': False,
    'epochs': 50,
    'lr': 2e-5, # default `2e-5`
    'patience': 5,
    'seed': 23
}
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/config.json', 'w') as file:
    json.dump(CONFIG, file)

def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

seed_all(CONFIG['seed'])

## Data

In [ ]:
with open(f'{DATA_PATH}/{CONFIG["data_file"]}') as file:
    data = json.load(file)

print('total records:', len(data))

In [ ]:
def last_ann(anns):
    return sorted(anns, key=lambda d: d['updated_at'])[-1]

ds = []
for d in data:
    dd = {}
    dd['idart'] = d['id']
    dd['anno'] = d['data']['anno']
    dd.update(
        {
            x['from_name'] + '_' + x['value']['choices'][0] : 1
            for x in last_ann(d['annotations'])['result']
        }
    )
    ds.append(dd)
df = pd.DataFrame(ds)
df.fillna(0, inplace=True)
seq_len = [len(str(i).split()) for i in df['anno']]
print('max sequence lenght:', max(seq_len))
display(df.head())

In [ ]:
def chk_balance(df, target_cols):
    print('total:', len(df))
    for col in target_cols:
        if 'target' in col: print(
            col, '\t',
            df[col].sum(), '\t',
            '{:.1%}'.format(df[col].sum() / len(df))
        )

In [ ]:
target_cols = sorted(list(set([
    c.replace('_explicit', '').replace('_general', '') 
    for c in CONFIG['targets']
])))
if CONFIG['targets_type'] == 'explicit':
    for col in target_cols:
        df[col] = df[col + '_explicit']
elif CONFIG['targets_type'] == 'general':
    for col in target_cols:
        name1 = col + '_explicit'
        name2 = col + '_general'
        if col == 'target_0':
            df[col] = df[name1]
        else:
            df[col] = df.apply(lambda x: max(x[name1], x[name2]), axis=1)
elif CONFIG['targets_type'] == 'all':
    target_cols = [col for col in df.columns if 'target' in col]
else:
    ValueError('`targets_type` parameter error')
if CONFIG['targets_type'] != 'all':
    df.loc[
        df[[c for c in target_cols if 'target_0' not in c]].sum(axis=1) == 0, 
        'target_0'
    ] = 1
chk_balance(df, target_cols)

In [ ]:
CONFIG['target_cols'] = target_cols
with open(f'{MDLS_PATH}/config.json', 'w') as file:
    json.dump(CONFIG, file)

In [ ]:
skf = StratifiedKFold(CONFIG['folds'], shuffle=True, random_state=CONFIG['seed'])
df['fold'] = -1
for i, (train_idxs, val_idxs) in enumerate(skf.split(df, df['target_0_explicit'])):
    df.loc[val_idxs, 'fold'] = i
for fold_num in range(CONFIG['folds']): 
    train_idxs = np.where((df['fold'] != fold_num))[0]
    val_idxs = np.where((df['fold'] == fold_num))[0]
    df_train = df.loc[train_idxs]
    df_val = df.loc[val_idxs]
    print('FOLD', fold_num)
    chk_balance(df_train, target_cols)
    chk_balance(df_val, target_cols)
    print('-' * 30)

In [ ]:
target_cols

## Training

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self, model_name, target_cols):
        super(BERTClass, self).__init__()
        self.rubert = AutoModel.from_pretrained(model_name)
        #self.l2 = torch.nn.Dropout(.3)
        self.fc = torch.nn.Linear(768, len(target_cols))
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.rubert(
            ids, 
            attention_mask=mask, 
            token_type_ids=token_type_ids, 
            return_dict=False
        )
        #output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

In [ ]:
class ArticlesDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_cols):
        self.df = df
        self.max_len = max_len
        self.text = df['anno']
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values if target_cols else []
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
class ArticlesTrainer:
    def __init__(self, model, device, 
                 optimizer, scheduler, 
                 criterion, acc_flag=True):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.criterion = criterion
        self.acc_flag = acc_flag
        if acc_flag:
            self.best_val_acc = 0
        else:
            self.best_val_loss = np.inf
        self.val_losses = []
        self.train_losses = []
        self.val_acc = []
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, val_loader, 
            max_patience, save_mode='best', save_name='model'):     
        n_patience = 0
        for n_epoch in range(1, epochs + 1):
            self.info_message('EPOCH: {}', n_epoch)
            train_loss, train_time = self.train_epoch(train_loader)
            val_loss, val_acc, val_f1_mic, val_f1_mac, val_time = self.val_epoch(val_loader)
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.val_acc.append(val_acc)
            self.info_message(
                'epoch train: {} | loss: {:.2f} | time: {:.0f} sec',
                n_epoch, train_loss, train_time
            )
            self.info_message(
                'epoch val: {} | loss: {:.2f} | ' +
                'acc: {:.2f} | f1 micro: {:.2f} | f1 macro: {:.2f} | ' +
                'time: {:.0f} sec',
                n_epoch, val_loss, val_acc, val_f1_mic, val_f1_mac, val_time
            )
            if self.acc_flag:
                if self.best_val_acc < val_acc: 
                    self.save_model(n_epoch, save_mode, save_name, val_loss, val_acc)
                    self.info_message(
                        'val accuracy improved {:.2f} -> {:.2f} | saved model to "{}"', 
                        self.best_val_acc, val_acc, self.lastmodel
                    )
                    self.best_val_acc = val_acc
                    n_patience = 0
                else:
                    n_patience += 1
            else:
                if self.best_val_loss > val_loss: 
                    self.save_model(n_epoch, save_mode, save_name, val_loss, val_acc)
                    self.info_message(
                        'val loss improved {:.2f} -> {:.2f} | saved model to "{}"', 
                        self.best_val_loss, val_loss, self.lastmodel
                    )
                    self.best_val_loss = val_loss
                    n_patience = 0
                else:
                    n_patience += 1
            if n_patience >= max_patience:
                self.info_message(
                    '\nno improvement for last {} epochs', 
                    n_patience
                )
                break
        history = {
            'train losses': self.train_losses, 
            'val losses': self.val_losses, 
            'val accuracy': self.val_acc
        }
        return history
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0
        for step, batch in enumerate(train_loader, 1):
            ids = batch['ids'].to(self.device, dtype=torch.long)
            mask = batch['mask'].to(self.device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(self.device, dtype=torch.long)
            targets = batch['targets'].to(self.device, dtype=torch.float)
            self.optimizer.zero_grad() 
            outputs = self.model(ids, mask, token_type_ids)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()
            if self.scheduler: 
                self.scheduler.step()
            sum_loss += loss.detach().item()   
            self.info_message(
                'train step {}/{} | train loss: {:.4f}           ',
                step, len(train_loader), sum_loss / step, end='\r'
            )
        return sum_loss / len(train_loader), int(time.time() - t)
    
    def val_epoch(self, val_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []
        for step, batch in enumerate(val_loader, 1):
            with torch.no_grad():
                ids = batch['ids'].to(self.device, dtype=torch.long)
                mask = batch['mask'].to(self.device, dtype=torch.long)
                token_type_ids = batch['token_type_ids'].to(self.device, dtype=torch.long)
                targets = batch['targets'].to(self.device, dtype=torch.float)
                outputs = self.model(ids, mask, token_type_ids)
                loss = self.criterion(outputs, targets)
                sum_loss += loss.detach().item()
                y_all.extend(targets.cpu().detach().numpy().tolist())
                outputs_all.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            self.info_message(
                'val step {}/{} | val loss: {:.4f}               ', 
                step, len(val_loader), sum_loss / step, end='\r'
            )
        outputs_all = np.array(outputs_all) > .5
        acc = accuracy_score(y_all, outputs_all)
        f1_mic = f1_score(y_all, outputs_all, average='micro')
        f1_mac = f1_score(y_all, outputs_all, average='macro')
        return sum_loss / len(val_loader), acc, f1_mic, f1_mac, int(time.time() - t)
    
    def save_model(self, n_epoch, save_mode, save_name, loss, acc):
        if save_mode == 'best':
            self.lastmodel = f'{MDLS_PATH}/{save_name}.pth'
        else:
            self.lastmodel = f'{MDLS_PATH}/{save_name}-e{n_epoch}-loss{loss:.3f}-acc{acc:.3f}.pth'
        dict_save = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'n_epoch': n_epoch,
        }
        if self.acc_flag:
            dict_save['best_val_acc'] = self.best_val_acc
        else:
            dict_save['best_val_loss'] = self.best_val_loss
        torch.save(dict_save, self.lastmodel)
    
    def display_plots(self):
        fig, axes = plt.subplots(figsize=(16, 4), nrows=1, ncols=2)
        axes[0].set_title(f'training and validation losses')
        axes[0].plot(self.val_losses, label='val')
        axes[0].plot(self.train_losses, label='train')
        axes[0].set_xlabel('iterations')
        axes[0].set_ylabel('loss')
        axes[0].legend()
        axes[1].set_title(f'validation accuracy')
        axes[1].plot(self.val_acc, label='val')
        axes[1].set_xlabel('iterations')
        axes[1].set_ylabel('accuracy')
        axes[1].legend()
        plt.show()
        plt.close()
    
    @staticmethod
    def info_message(message, *args, end='\n'):
        print(message.format(*args), end=end)

In [ ]:
def train_art_model(df_train, df_val, target_cols,
                    device, model_name, max_seq_len,
                    epochs, save_name, patience, 
                    batch_size, num_workers):
    print('=' * 20, f'MODEL TRAIN - {save_name}', '=' * 20)
    print('train:', df_train.shape, '| val:', df_val.shape)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    train_dataset = ArticlesDataset(
        df=df_train, 
        tokenizer=tokenizer, 
        max_len=max_seq_len, 
        target_cols=target_cols
    )
    val_dataset = ArticlesDataset(
        df=df_val, 
        tokenizer=tokenizer, 
        max_len=max_seq_len, 
        target_cols=target_cols
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=batch_size,
        num_workers=num_workers, 
        shuffle=True,
        pin_memory=True
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        num_workers=num_workers, 
        shuffle=False, 
        pin_memory=True
    )
    model = BERTClass(
        model_name=model_name, 
        target_cols=target_cols
    )
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=1e-6)
    scheduler = None
    criterion = nn.functional.binary_cross_entropy_with_logits
    trainer = ArticlesTrainer(
        model, 
        device, 
        optimizer, 
        scheduler,
        criterion,
        acc_flag=CONFIG['acc']
    )
    history = trainer.fit(
        epochs, 
        train_loader, 
        val_loader, 
        save_mode='best', 
        save_name=save_name,
        max_patience=patience
    )
    trainer.display_plots()
    with open(f'{MDLS_PATH}/history_{save_name}.json', 'w') as file:
        json.dump(history, file)
    return trainer.lastmodel

In [ ]:
model_files = []
for fold_num in range(CONFIG['folds']): 
    train_idxs = np.where((df['fold'] != fold_num))[0]
    val_idxs = np.where((df['fold'] == fold_num))[0]
    df_train = df.loc[train_idxs]
    df_val = df.loc[val_idxs]
    df_train.reset_index(drop=True, inplace=True)
    df_val.reset_index(drop=True, inplace=True)
    model_files.append(train_art_model(
        df_train, 
        df_val, 
        target_cols,
        device=CONFIG['device'], 
        model_name=CONFIG['bbone'],
        max_seq_len=CONFIG['max_seq_len'],
        epochs=CONFIG['epochs'], 
        save_name=f'model_{fold_num}',
        patience=CONFIG['patience'], 
        batch_size=CONFIG['batch_size'],
        num_workers=CONFIG['num_workers']
    ))
print(model_files)
with open(f'{MDLS_PATH}/model_files.json', 'w') as file:
    json.dump(model_files, file)

## Inference

In [ ]:
with open(f'{MDLS_PATH}/model_files.json', 'r') as file:
    model_files = json.load(file)
print("models files loaded:", model_files)

In [ ]:
def infer(model_file, df, target_cols, 
          model_name, max_seq_len,
          device, batch_size, num_workers):
    print('PREDICT:', model_file, df.shape)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    pred_dataset = ArticlesDataset(
        df=df, 
        tokenizer=tokenizer, 
        max_len=max_seq_len, 
        target_cols=target_cols
    )
    pred_loader = torch.utils.data.DataLoader(
        pred_dataset, 
        batch_size=batch_size, 
        num_workers=num_workers, 
        shuffle=False, 
        pin_memory=True
    )
    model = BERTClass(
        model_name=model_name, 
        target_cols=target_cols
    )
    model.to(device)
    checkpoint = torch.load(model_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    y_pred = []
    for i, batch in enumerate(pred_loader, start=1):
        print(f'prediction step {i}/{len(pred_loader)}   ', end='\r')
        with torch.no_grad():
            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            tmp_pred = model(ids, mask, token_type_ids)
            y_pred.extend(torch.sigmoid(tmp_pred).cpu().detach().numpy().tolist())
    df_pred = pd.DataFrame(y_pred) 
    df_pred.columns = [c + '_pred' for c in target_cols]
    return df_pred

In [ ]:
df_pred = pd.DataFrame()
for model_file in tqdm(model_files):
    df_pred_tmp = infer(
        model_file=model_file, 
        df=df_val, 
        target_cols=target_cols,
        model_name=CONFIG['bbone'],
        max_seq_len=CONFIG['max_seq_len'],
        device=CONFIG['device'], 
        batch_size=CONFIG['batch_size'],
        num_workers=CONFIG['num_workers']
    )
    if len(df_pred):
        df_pred += df_pred_tmp #.add(df_pred_tmp, fill_value=0)
    else:
        df_pred = df_pred_tmp
df_pred /= len(model_files)
display(df_pred.head())

In [ ]:
y_unique = target_cols
mcm = multilabel_confusion_matrix(
    (df_pred > .5).values, 
    df_val[target_cols].values
)
for c, cm in zip(target_cols, mcm):
    print(
        c, '\t',
        df_val[c].sum(), '\t',
            '{:.1%}'.format(df[c].sum() / len(df)),
    )
    print(cm)
    print('=' * 50)

In [ ]:
df_val.head()

In [ ]:
text = """Цель настоящего исследования заключается в изучении рисков от изменения климата в 
городской системе Стамбула. Стамбул является крупнейшим городом в Турции по численности
населения и наличию экономических возможностей, в связи с чем любой связанный с климатом 
риск будет разрушительным не только для города, но и для страны в целом. Городская система 
определяется на основе городских секторов, которые являются центрами деловой активности, 
управления, экологических систем, ресурсов и имеют решающее значение для жизнеспособности 
экономики и общественного здоровья города. Это также делает их уязвимыми перед климатическими 
катастрофами. С учетом стратегий развития Стамбула в области водных ресурсов, 
здравоохранения, энергетики, сельского хозяйства, транспорта, развития и землепользования, 
общественной безопасности, инфраструктуры, биоразнообразия и экологии, культуры, 
материалов определены 11 городских секторов и 25 подсекторов. Руководство МСМЭИ (ICLEI-Local 
Governments for Sustainability - Международный совет по местным экологическим инициативам) 
редоставило инструкцию по оценке рисков для этих территорий и секторов «Подготовка 
к изменению климата: руководство для местных, региональных и национальных правительств». 
Данные получены в ходе глубинных интервью с городскими стейкхолдерами. Секторы ранжированы 
с учетом факторов риска для каждого из них. Результаты исследования позволили выявить 
городские секторы, подверженные большему и меньшему риску из-за последствий изменения климата. 
Определение рисков от изменения климата в жизненно важных секторах Стамбула является 
необходимым при принятии решений для разработки дальнейших стратегий по смягчению возможных 
последствий и адаптации к новым условиям."""
text = df_val.loc[1, "anno"]
print(text[:50], len(text))

In [ ]:
d = {'anno': [text]}
d.update(dict(zip(target_cols, [0] * len(target_cols))))
df_txt = pd.DataFrame(d)
df_txt

In [ ]:
%%time
df_pred_tmp = infer(
    model_file=model_files[0], 
    df=df_txt, 
    target_cols=target_cols,
    model_name=CONFIG['bbone'],
    max_seq_len=CONFIG['max_seq_len'],
    device=CONFIG['device'], 
    batch_size=CONFIG['batch_size'],
    num_workers=CONFIG['num_workers']
)
df_pred_tmp